In [ ]:
import nest_asyncio
import uvicorn
import asyncio
from fastapi import FastAPI, Query
from fastapi.responses import HTMLResponse
import json
import re

# Apply compatibility for Jupyter Notebook
nest_asyncio.apply()

# Function to Convert TXT to JSON
def convert_txt_to_json(txt_file="legislation.txt", json_file="book.json"):
    data = []
    current_article = None

    with open(txt_file, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()

            # Detect "Article X:" format and create an article entry
            if re.match(r"^Article \d+:", line):
                current_article = {
                    "article_number": line.split(":")[0].replace("Article ", ""),
                    "title": line.split(":")[1].strip(),
                    "sections": []
                }
                data.append(current_article)

            # Detect "X.X" sub-sections
            elif re.match(r"^\d+\.\d+", line) and current_article:
                section_number = line.split(" ")[0]
                section_text = " ".join(line.split(" ")[1:])
                current_article["sections"].append({"section_number": section_number, "text": section_text})

    # Save structured JSON format
    with open(json_file, "w", encoding="utf-8") as json_file:
        json.dump({"title": "Senate Legislation Aksum University 2025", "articles": data}, json_file, indent=4)

# Convert TXT to JSON before starting the app
convert_txt_to_json()

# Load JSON document
def load_legislation(filename="book.json"):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

BOOK_CONTENT = load_legislation()

# Initialize FastAPI App
app = FastAPI(title="University Legislation Search App")

@app.get("/", response_class=HTMLResponse)
def home():
    """Render a simple search interface."""
    return """
    <html>
        <head>
            <title>University Legislation Search</title>
            <style>
                body { font-family: Arial, sans-serif; text-align: center; }
                input { padding: 8px; width: 300px; }
                form { margin-top: 20px; }
            </style>
        </head>
        <body>
            <h1>University Legislation Search</h1>
            <form action="/search" method="get">
                <input type="text" name="q" placeholder="Search by keyword, article, or text..." />
                <input type="submit" value="Search" />
            </form>
        </body>
    </html>
    """

@app.get("/search", response_class=HTMLResponse)
def search(q: str = Query(..., min_length=1)):
    """Search university legislation based on user input."""
    q_lower = q.lower().strip()
    results = []

    # Search through articles and sub-articles
    for article in BOOK_CONTENT["articles"]:
        if q_lower in article["title"].lower() or q_lower in article["article_number"]:
            results.append(f"<b>Article {article['article_number']}: {article['title']}</b><br>")
            if "sections" in article:
                for section in article["sections"]:
                    results.append(f"{section['section_number']}. {section['text']}<br>")
        if "sub_articles" in article:
            for sub_article in article["sub_articles"]:
                if q_lower in sub_article["title"].lower():
                    results.append(f"<b>Sub-Article {sub_article['sub_article_number']}: {sub_article['title']}</b><br>")
                    results.append(f"{sub_article['definition']}<br>")

    # Construct HTML response with formatted text
    html = """
    <html>
        <head>
            <title>Search Results</title>
            <style>
                body { font-family: Arial, sans-serif; }
                ul { padding: 10px; }
                li { margin-bottom: 10px; }
                pre { white-space: pre-wrap; font-size: 16px; line-height: 1.5; }
            </style>
        </head>
        <body>
            <h1>Search Results</h1>
            <a href="/">Back to Search</a>
            <ul>
    """

    html += "".join(f"<li>{entry}</li>" for entry in results) if results else "<li>No results found.</li>"

    html += """
            </ul>
        </body>
    </html>
    """
    return html

# Function to start FastAPI server inside Jupyter
async def start_app():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()

# Run the server asynchronously inside Jupyter
asyncio.run(start_app())


INFO:     Started server process [20844]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57335 - "GET / HTTP/1.1" 200 OK
